In [22]:
# Cell 1 – Paths, DuckDB session, view setup
from pathlib import Path

import duckdb
import pandas as pd

PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_PATH = PROJECT_ROOT / "data" / "processed"
SQL_DIR = PROJECT_ROOT / "references" / "sql"
REPORTS_DIR = PROJECT_ROOT / "reports"
FIG_DIR = REPORTS_DIR / "figures"
TAB_DIR = REPORTS_DIR / "tables"

for path in [REPORTS_DIR, FIG_DIR, TAB_DIR]:
    path.mkdir(parents=True, exist_ok=True)

con = duckdb.connect()

clean_csv = DATA_PATH / "clean_data.csv"
if not clean_csv.exists():
    raise FileNotFoundError("cant find data/processed/clean_data.csv")

con.execute(f"""
    CREATE OR REPLACE VIEW users AS
    SELECT
      TRY_CAST(TRIM(userid) AS BIGINT)                              AS userid,
      TRIM(version)                                                 AS version,
      TRY_CAST(TRIM(session_count) AS INTEGER)                      AS session_count,
      COALESCE(TRY_CAST(NULLIF(TRIM(retention_1), '') AS BOOLEAN), FALSE) AS retention_1,
      COALESCE(TRY_CAST(NULLIF(TRIM(retention_7), '') AS BOOLEAN), FALSE) AS retention_7,
      TRIM(acquisition_channel)                                     AS acquisition_channel,
      TRIM(country)                                                 AS country,
      TRIM(platform)                                                AS platform,
      COALESCE(TRY_CAST(NULLIF(TRIM(purchase), '') AS INTEGER), 0)  AS purchase,
      COALESCE(TRY_CAST(NULLIF(TRIM("CAC"), '') AS DOUBLE), 0.0)    AS cac,
      COALESCE(TRY_CAST(NULLIF(TRIM(revenue), '') AS DOUBLE), 0.0)  AS revenue,
      COALESCE(TRY_CAST(NULLIF(TRIM("ROI"), '') AS DOUBLE), 0.0)    AS roi_row
    FROM read_csv_auto(
      '{clean_csv.as_posix()}',
      HEADER=TRUE,
      SAMPLE_SIZE=-1,
      ALL_VARCHAR=TRUE
    );
""")

# Alias needed for SQL files that expect the events table
con.execute("CREATE OR REPLACE VIEW events AS SELECT * FROM users;")

preview = con.execute("SELECT * FROM users LIMIT 5").df()
types = con.execute(
    """
    SELECT
      typeof(userid)        AS t_userid,
      typeof(session_count) AS t_session_count,
      typeof(retention_1)   AS t_ret1,
      typeof(retention_7)   AS t_ret7,
      typeof(purchase)      AS t_purchase,
      typeof(cac)           AS t_cac,
      typeof(revenue)       AS t_revenue,
      typeof(roi_row)       AS t_roi_row
    FROM users
    LIMIT 1
    """
).df()

display(preview)
display(types)

,userid,version,session_count,retention_1,retention_7,acquisition_channel,country,platform,purchase,cac,revenue,roi_row
0,116,gate_30,3,False,False,Facebook,USA,Google Play,0,2.8,0.023765,-0.991512
1,337,gate_30,38,False,False,TikTok,USA,Google Play,0,1.7,0.062804,-0.963057
2,377,gate_40,165,True,False,Facebook,USA,Google Play,0,2.8,0.087634,-0.968702
3,483,gate_40,1,False,False,Facebook,Mexico,Google Play,0,2.8,0.011883,-0.995756
4,488,gate_40,179,False,True,TikTok,USA,App Store,0,1.7,0.771525,-0.546162


,t_userid,t_session_count,t_ret1,t_ret7,t_purchase,t_cac,t_revenue,t_roi_row
0,BIGINT,INTEGER,BOOLEAN,BOOLEAN,INTEGER,DOUBLE,DOUBLE,DOUBLE


In [23]:
# Cell 2 – Run canonical SQL queries
q_installs = (SQL_DIR / "installs_by_channel.sql").read_text(encoding="utf-8")
q_funnel   = (SQL_DIR / "funnel_step_rates.sql").read_text(encoding="utf-8")
q_roi      = (SQL_DIR / "roi_by_channel.sql").read_text(encoding="utf-8")
q_funnel_by_channel = (SQL_DIR / "funnel_by_channel.sql").read_text(encoding="utf-8")

df_installs = con.execute(q_installs).df()
df_funnel   = con.execute(q_funnel).df()
df_roi      = con.execute(q_roi).df()
df_funnel_channel = con.execute(q_funnel_by_channel).df()

display(df_installs.head())
display(df_funnel)
display(df_roi.head())
display(df_funnel_channel.head())


,acquisition_channel,users
0,Instagram,36281
1,Facebook,26776
2,TikTok,17979
3,Organic,9153


,n_users,rate_install,rate_d1_from_install,rate_d7_from_d1,rate_purchase_from_d7
0,90189,1.0,0.454867,0.731962,0.16751


,acquisition_channel,users,revenue,ad_spend,roi,roas
0,Organic,9153,5091.083966,2745.9,0.854068,1.854068
1,TikTok,17979,10415.219151,30564.3,-0.659236,0.340764
2,Instagram,36281,19274.536869,83446.3,-0.769019,0.230981
3,Facebook,26776,14523.342010,74972.8,-0.806285,0.193715


,acquisition_channel,n_users,rate_d1_from_install,rate_d7_from_d1,rate_purchase_overall
0,Organic,9153,0.476674,0.738253,0.055064
1,Instagram,36281,0.459855,0.734356,0.056421
2,Facebook,26776,0.448947,0.728558,0.055535
3,TikTok,17979,0.442516,0.728632,0.055175


In [24]:
# Cell 3 – Export tables to avoid clobbering notebook outputs
(df_installs
 .rename(columns=str.lower)
 .to_csv(TAB_DIR / "sql_installs_by_channel.csv", index=False))

(df_funnel
 .rename(columns=str.lower)
 .to_csv(TAB_DIR / "sql_funnel_step_rates.csv", index=False))

(df_roi
 .rename(columns=str.lower)
 .to_csv(TAB_DIR / "sql_roi_by_channel.csv", index=False))

(df_funnel_channel
 .rename(columns=str.lower)
 .to_csv(TAB_DIR / "sql_funnel_by_channel.csv", index=False))

print("Saved SQL outputs to reports/tables/sql_*.csv")

Saved SQL outputs to reports/tables/sql_*.csv


In [25]:
# Cell 4 – Optional cross-check against notebook outputs
nb_funnel = pd.read_csv(TAB_DIR / "funnel.csv")
diff = df_funnel.round(6) - nb_funnel.round(6)

print("SQL vs notebook funnel comparison (rounded 6 decimals):")
display(diff)

SQL vs notebook funnel comparison (rounded 6 decimals):


,n_users,rate_d1_from_install,rate_d1_from_onboarding,rate_d7_from_d1,rate_install,rate_onboarding_from_install,rate_purchase_from_d7,rate_purchase_overall
0,0,NaN,NaN,0.0,0.0,NaN,0.0,NaN
